In [1]:
import urllib3
import certifi
import json
import sqlite3
import pandas as pd

In [2]:
def source_data_from_parquet(parquet_file_name):
    try:
        df_parquet = pd.read_parquet(parquet_file_name)
    except Exception as e:
        print(f"Error reading parquet file: {e}")
        df_parquet = pd.DataFrame()
    return df_parquet


def source_data_from_csv(csv_file_name):
    try:
        df_csv = pd.read_csv(csv_file_name)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        df_csv = pd.DataFrame()
    return df_csv


def source_data_from_api(api_endpoint):
    try:
        # Create a PoolManager to handle HTTP requests
        http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
        api_response = http.request('GET', api_endpoint)
        apt_status = api_response.status
        if apt_status == 200:
            data = json.loads(api_response.data.decode('utf-8'))
            df_api = pd.json_normalize(data)
        else:
            print(f"API call failed with status code: {apt_status}")
            df_api = pd.DataFrame()
    except Exception as e:
        print(f"Error retrieving data from API: {e}")
        df_api = pd.DataFrame()
    return df_api


def source_data_from_table(db_name, table_name):
    try:
        # Read SQLite query results into a pandas DataFrame
        with sqlite3.connect(db_name) as conn:
            df_table = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    except Exception as e:
        print(f"Error reading data from table {table_name}: {e}")
        df_table = pd.DataFrame()
    return df_table


def source_data_from_webpage(web_page_url, matching_keyword):
    try:
        # Read webpage table into a pandas DataFrame
        df_html = pd.read_html(web_page_url, match=matching_keyword)
        df_html = df_html[0]  # Get the first matching table
    except Exception as e:
        print(f"Error reading data from webpage: {e}")
        df_html = pd.DataFrame()
    return df_html

In [5]:
def extract_data():
    parquet_file_name = "data/yellow_tripdata_2022-01.parquet"
    csv_file_name = "data/h9gi-nx95.csv"
    api_endpoint = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500"
    db_name = "movies.sqlite"
    table_name = "movies"
    web_page_url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    matching_keyword = "by country"

    # Extract data from all source systems
    # These dataframes are available for loading data into either a staging area or transformation pipelines.
    df_parquet, df_csv, df_api, df_table, df_html = (
        source_data_from_parquet(parquet_file_name),
        source_data_from_csv(csv_file_name),
        source_data_from_api(api_endpoint),
        source_data_from_table(db_name, table_name),
        source_data_from_webpage(web_page_url, matching_keyword),
    )
    return df_parquet, df_csv, df_api, df_table, df_html

In [6]:
# Call the function to test
df_parquet, df_csv, df_api, df_table, df_html = extract_data()

In [7]:
# Print a preview of the extracted data
print("Parquet DataFrame:")
print(df_parquet.head())
print("\nCSV DataFrame:")
print(df_csv.head())
print("\nAPI DataFrame:")
print(df_api.head())
print("\nTable DataFrame:")
print(df_table.head())
print("\nHTML DataFrame:")
print(df_html.head())

Parquet DataFrame:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           166           166   
3           1.09         1.0                  N           114            68   
4           4.30         1.0                  N            68           163   

   payment_type  fare_amount  extra  mta_tax  tip_amoun